In [ ]:
import os, sys, json, gc, warnings
from pathlib import Path

# Core data tools
import numpy as np
import pandas as pd

# PyTorch
import torch
import torch.nn.functional as F

# Hugging Face
import transformers
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    default_data_collator,
)
from datasets import Dataset
from evaluate import load as load_metric  # optional

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, f1_score

# Audio
import librosa
import soundfile as sf
from pydub import AudioSegment
from pydub.utils import which
import music21

# UI
import streamlit as st

# pydub ↔ ffmpeg (use system/conda ffmpeg if available)
AudioSegment.converter = which("ffmpeg") or AudioSegment.converter

# Quiet a few noisy-but-harmless warnings
warnings.filterwarnings("ignore", message=".*Triton.*")
warnings.filterwarnings("ignore", message=".*weight_norm is deprecated.*")
warnings.filterwarnings("ignore", message="pkg_resources is deprecated.*")

# Version sanity checks (comment out if you prefer flexibility)
assert torch.__version__.startswith("2.1.0"), f"Torch pin expected 2.1.0, got {torch.__version__}"
assert transformers.__version__.startswith("4.37."), f"Transformers pin expected 4.37.x, got {transformers.__version__}"
import numpy  # keep separate to check the exact package version below
assert numpy.__version__ == "1.26.4", f"Numpy pin expected 1.26.4, got {numpy.__version__}"


In [5]:
data_path = "C:/SonicAid_clean/data/raw"
# Load GoEmotions parts
geo1 = pd.read_csv(os.path.join(data_path, "goemotions_1.csv"))
geo2 = pd.read_csv(os.path.join(data_path, "goemotions_2.csv"))
geo3 = pd.read_csv(os.path.join(data_path, "goemotions_3.csv"))

# Preview each
print(f"GoEmotions 1: {geo1.shape}\n", geo1.head(2), "\n")
print(f"GoEmotions 2: {geo2.shape}\n", geo2.head(2), "\n")
print(f"GoEmotions 3: {geo3.shape}\n", geo3.head(2), "\n")

# Confirm columns are the same
print("Column consistency check:", geo1.columns.equals(geo2.columns) and geo1.columns.equals(geo3.columns))
print("Columns:", geo1.columns.tolist())

# Combine all three
goemotions_df = pd.concat([geo1, geo2, geo3], ignore_index=True)
print(" Combined GoEmotions shape:", goemotions_df.shape)
print(goemotions_df[['text'] + list(goemotions_df.columns[-10:])].sample(3))

GoEmotions 1: (70000, 37)
                                                 text       id       author  \
0                                    That game hurt.  eew5j0j        Brdd9   
1   >sexuality shouldn’t be a grouping category I...  eemcysk  TheGreen888   

          subreddit    link_id   parent_id   created_utc  rater_id  \
0               nrl  t3_ajis4z  t1_eew18eq  1.548381e+09         1   
1  unpopularopinion  t3_ai4q37   t3_ai4q37  1.548084e+09        37   

   example_very_unclear  admiration  ...  love  nervousness  optimism  pride  \
0                 False           0  ...     0            0         0      0   
1                  True           0  ...     0            0         0      0   

   realization  relief  remorse  sadness  surprise  neutral  
0            0       0        0        1         0        0  
1            0       0        0        0         0        0  

[2 rows x 37 columns] 

GoEmotions 2: (70000, 37)
                              text       id      

In [ ]:

os.makedirs("C:/SonicAid_clean/data/raw", exist_ok=True)

# ensure boolean type for this flag
if goemotions_df["example_very_unclear"].dtype == object:
    goemotions_df["example_very_unclear"] = goemotions_df["example_very_unclear"].astype(str).str.lower().map({"true": True, "false": False})

# drop unclear rows
goemotions_df = goemotions_df[goemotions_df["example_very_unclear"] == False].reset_index(drop=True)

# detect emotion columns: columns that are all in {0,1,True,False} (ignoring NA)
def is_binary_series(s):
    s2 = s.dropna()
    return s2.isin([0,1,True,False]).all()

candidate_cols = [c for c in goemotions_df.columns if is_binary_series(goemotions_df[c])]
# common non-emotion binaries to exclude
exclude = {"example_very_unclear"}
emotion_columns = [c for c in candidate_cols if c not in exclude]

# coerce to numeric 0/1
goemotions_df[emotion_columns] = goemotions_df[emotion_columns].apply(pd.to_numeric, errors="coerce").fillna(0).astype(int)

# keep rows with ≥1 emotion
mask_has_emotion = goemotions_df[emotion_columns].sum(axis=1) > 0
goemotions_df = goemotions_df[mask_has_emotion].reset_index(drop=True)

# single-label: take the first active emotion (or argmax)
goemotions_df["label"] = goemotions_df[emotion_columns].idxmax(axis=1)

# final tidy
goemotions_clean = goemotions_df[["text", "label"]].dropna().reset_index(drop=True)

print("Cleaned GoEmotions sample:")
print(goemotions_clean.sample(min(5, len(goemotions_clean))))
print("Shape:", goemotions_clean.shape)

goemotions_clean.to_csv("C:/SonicAid_clean/data/raw/goemotions_df.csv", index=False)

#  Clean Dreaddit Dataset

# Load train/test files
df_train = pd.read_csv("C:/SonicAid_clean/data/raw/dreaddit-train.csv")
df_test = pd.read_csv("C:/SonicAid_clean/data/raw/dreaddit-test.csv")

# Combine and filter
dreaddit_full = pd.concat([df_train, df_test], ignore_index=True)[["text", "label"]]
dreaddit_full.dropna(subset=["text", "label"], inplace=True)

# Map to readable labels
dreaddit_full["label"] = dreaddit_full["label"].map({1: "distress", 0: "no_distress"})

# Final check
print("Cleaned Dreaddit sample:")
print(dreaddit_full.sample(5))
print("Shape:", dreaddit_full.shape)

# Save cleaned file
dreaddit_full.to_csv("C:/SonicAid_clean/data/raw/dreaddit_df.csv", index=False)


✅ Cleaned GoEmotions sample:
                                                    text      label
158821        [NAME] still looks fresh for 40 years old.  gratitude
103196        They just keep coming, it's like a cartoon    neutral
197092     TYM is the best place for that kind of thing.    neutral
1528    Water? Find a way to keep your temperature down.     caring
101954   It did. MM used way more textures than Ocarina.   approval
Shape: (207814, 2)
✅ Cleaned Dreaddit sample:
                                                   text        label
3318  I don't remember always being like this, but o...  no_distress
3125  Are there federal or state (IN) laws that gove...  no_distress
2696  I just feel kinda gross because I was giving h...     distress
47    Dental Lifeline Network's Donated Dental Servi...  no_distress
3053  I’m not afraid of this guy, at all, and I have...     distress
Shape: (3553, 2)


In [ ]:

goemotions_df = pd.read_csv("data/raw/goemotions_df.csv")
goemotions_df['label'] = goemotions_df['label'].astype('category')
goemotions_df['label_id'] = goemotions_df['label'].cat.codes

label2id = dict(enumerate(goemotions_df['label'].cat.categories))
id2label = {v: k for k, v in label2id.items()}

goemotions_dataset = Dataset.from_pandas(
goemotions_df[['text', 'label_id']].rename(columns={'label_id': 'label'})
)

checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(
        checkpoint,
        num_labels=len(label2id),
        id2label=label2id,
        label2id=id2label
).to("cuda")

print("Using device:", model.device)

def tokenize(batch):
        return tokenizer(batch["text"], padding=True, truncation=True)

tokenized_dataset = goemotions_dataset.map(tokenize, batched=True, remove_columns=["text"])
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.1)

def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        preds = np.argmax(predictions, axis=1)
        return {
            "accuracy": accuracy_score(labels, preds),
            "f1": f1_score(labels, preds, average="weighted")
        }

training_args = TrainingArguments(
        output_dir="./results_goemotions_v2",
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        learning_rate=2e-5,
        num_train_epochs=3,
        weight_decay=0.01,
        logging_dir="./logs_goemotions_v2",
        do_train=True,
        do_eval=True,
        logging_steps=50,
        save_steps=500
    )

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["test"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

trainer.train()

    # 8. Save the trained model
save_path = "./saved_models/goemotions_distilbert_v2"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"Model and tokenizer saved to: {save_path}")

# 8. Save the trained model and tokenizer
save_path = "./saved_models/goemotions_distilbert_v3"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model and tokenizer saved to: {save_path}")

In [7]:
save_path = "C:/SonicAid_clean/saved_models/goemotions_distilbert_v3"

loaded_tokenizer = AutoTokenizer.from_pretrained(save_path)

loaded_model = AutoModelForSequenceClassification.from_pretrained(save_path)


In [ ]:

dreaddit_df = pd.read_csv("C:/SonicAid_clean/data/raw/dreaddit_df.csv")[['text', 'label']].dropna()
dreaddit_df["label"] = dreaddit_df["label"].map({"no_distress": 0, "distress": 1})

# 2. Convert to HuggingFace Dataset
dreaddit_dataset = Dataset.from_pandas(dreaddit_df)

# 3. Tokenization
checkpoint = "distilbert-base-uncased"
tokenizer_dreaddit = AutoTokenizer.from_pretrained(checkpoint)

def tokenize(batch):
    return tokenizer_dreaddit(batch["text"], padding=True, truncation=True)

tokenized_dreaddit = dreaddit_dataset.map(tokenize, batched=True, remove_columns=["text"])
tokenized_dreaddit = tokenized_dreaddit.train_test_split(test_size=0.1)

# 4. Load model (binary classification) and move to GPU
model_dreaddit = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels=2
).to("cuda")

# 5. Define evaluation metrics
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.argmax(preds, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds)
    }

# 6. Training setup
training_args_dreaddit = TrainingArguments(
    output_dir="./results_dreaddit",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs_dreaddit"
)

trainer_dreaddit = Trainer(
    model=model_dreaddit,
    args=training_args_dreaddit,
    train_dataset=tokenized_dreaddit["train"],
    eval_dataset=tokenized_dreaddit["test"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer_dreaddit
)

# 7. Train the model
trainer_dreaddit.train()

# 8. Save model and tokenizer
save_path = "saved_models/distress_classifier"
trainer_dreaddit.save_model(save_path)
tokenizer_dreaddit.save_pretrained(save_path)
print(f"Distress model saved to: {save_path}")

# 9. Evaluate the model
metrics_dreaddit = trainer_dreaddit.evaluate()
print("Distress Model Evaluation Results:")
for k, v in metrics_dreaddit.items():
    print(f"{k}: {v:.4f}")

# 10. Optional inference example
sample_text = ["I'm feeling overwhelmed and anxious."]
inputs = tokenizer_dreaddit(sample_text, return_tensors="pt").to(model_dreaddit.device)
outputs = model_dreaddit(**inputs)
pred = torch.argmax(outputs.logits, dim=1)
print(f"redicted label: {'distress' if pred.item() == 1 else 'no_distress'}")


In [ ]:

# Optional: helps fragmentation on small GPUs (set BEFORE torch allocates memory)
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "max_split_size_mb:128")

# 1) Load data
dreaddit_df = pd.read_csv("C:/SonicAid_clean/data/raw/dreaddit_df.csv")[['text', 'label']].dropna()
dreaddit_df["label"] = dreaddit_df["label"].map({"no_distress": 0, "distress": 1}).astype(int)
dreaddit_dataset = Dataset.from_pandas(dreaddit_df)

# 2) Tokenizer  DO NOT pad here; set a lower max_length
checkpoint = "distilbert-base-uncased"
tokenizer_dreaddit = AutoTokenizer.from_pretrained(checkpoint)

def tokenize(batch):
    return tokenizer_dreaddit(batch["text"], truncation=True, max_length=128) 

tokenized = dreaddit_dataset.map(tokenize, batched=True, remove_columns=["text"])
tokenized = tokenized.train_test_split(test_size=0.1)

# Dynamic padding happens in the collator (saves lots of memory)
collator = DataCollatorWithPadding(tokenizer=tokenizer_dreaddit)

# 3) Model — disable cache to save memory during training
model_dreaddit = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
model_dreaddit.config.use_cache = False  # important for training memory
model_dreaddit.to("cuda")

# 4) Metrics
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.argmax(preds, axis=1)
    return {"accuracy": accuracy_score(labels, preds),
            "f1": f1_score(labels, preds)}

# 5) Training args smaller batches, fp16, grad checkpointing, eval accumulation
training_args_dreaddit = TrainingArguments(
    output_dir="./results_dreaddit",
    per_device_train_batch_size=4,       
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    gradient_checkpointing=True, 
    eval_accumulation_steps=2, 
    logging_dir="./logs_dreaddit",
    dataloader_num_workers=0,
)

trainer_dreaddit = Trainer(
    model=model_dreaddit,
    args=training_args_dreaddit,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer_dreaddit,
    data_collator=collator,
)

torch.cuda.empty_cache() 
trainer_dreaddit.train()

# Save & evaluate
save_path = "saved_models/distress_classifier"
trainer_dreaddit.save_model(save_path)
tokenizer_dreaddit.save_pretrained(save_path)
print(f"Distress model saved to: {save_path}")

metrics = trainer_dreaddit.evaluate()
print("Distress Model Evaluation Results:")
for k, v in metrics.items():
    try:
        print(f"{k}: {v:.4f}")
    except Exception:
        print(k, v)

# Inference example
sample_text = ["I'm feeling overwhelmed and anxious."]
inputs = tokenizer_dreaddit(sample_text, return_tensors="pt").to(model_dreaddit.device)
with torch.no_grad():
    outputs = model_dreaddit(**inputs)
pred = torch.argmax(outputs.logits, dim=1)
print(f"Predicted label: {'distress' if pred.item() == 1 else 'no_distress'}")



d:\Miniconda3\envs\sonicaid\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/3553 [00:00<?, ? examples/s]

NameError: name 'DataCollatorWithPadding' is not defined

In [ ]:

# 1. Load saved model and tokenizer
save_path = "C:/SonicAid_clean/saved_models/goemotions_distilbert_v3"

loaded_model = AutoModelForSequenceClassification.from_pretrained(save_path).to("cuda")
loaded_tokenizer = AutoTokenizer.from_pretrained(save_path)

# 2. Load and preprocess dataset
goemotions_df = pd.read_csv("data/raw/goemotions_df.csv")
goemotions_df["label"] = goemotions_df["label"].astype("category")
goemotions_df["label_id"] = goemotions_df["label"].cat.codes

goemotions_dataset = Dataset.from_pandas(
    goemotions_df[["text", "label_id"]].rename(columns={"label_id": "label"})
)

# 3. Tokenization
def tokenize(batch):
    return loaded_tokenizer(batch["text"], padding=True, truncation=True)

tokenized = goemotions_dataset.map(tokenize, batched=True, remove_columns=["text"])
tokenized = tokenized.train_test_split(test_size=0.1)

# 4. Define metrics
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.argmax(preds, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

# 5. Evaluation setup
eval_args = TrainingArguments(
    output_dir="./eval_results",
    per_device_eval_batch_size=16,
    do_eval=True,
    logging_dir="./logs_eval"
)

trainer_eval = Trainer(
    model=loaded_model,
    args=eval_args,
    eval_dataset=tokenized["test"],
    compute_metrics=compute_metrics,
    tokenizer=loaded_tokenizer
)

# 6. Run evaluation
results = trainer_eval.evaluate()

# 7. Display results
print("📊 GoEmotions Model Evaluation (Reloaded):")
for k, v in results.items():
    print(f"{k}: {v:.4f}")


In [ ]:

# 1. Load data
df = pd.read_csv("data/raw/goemotions_df.csv")
texts = df["text"].tolist()
labels = df["label"].tolist()

# 2. Label encoding
le = LabelEncoder()
encoded_labels = le.fit_transform(labels)
num_labels = len(le.classes_)

# 3. Build HF Dataset
dataset = Dataset.from_dict({"text": texts, "label": encoded_labels})
dataset = dataset.train_test_split(test_size=0.1)

# 4. Tokenizer and Model
checkpoint = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=num_labels)

# 5. Tokenization
def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

tokenized = dataset.map(tokenize, batched=True)

# 6. Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), dim=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

# 7. Training args
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./roberta-goemotion-run",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    do_eval=True,
    save_steps=500  
)


# 8. Trainer
model.to("cuda")
trainer_rob = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 9. Train!
trainer_rob.train()

In [ ]:
# Save fine-tuned model
trainer_rob.save_model("C:/SonicAid_clean/saved_models/roberta-goemotion-final")

# Save tokenizer (must match the model)
tokenizer.save_pretrained("C:/SonicAid_clean/saved_models/roberta-goemotion-final")


In [ ]:

# Load
model_path = "C:/SonicAid_clean/saved_models/roberta-goemotion-final"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load data
df = pd.read_csv("data/raw/goemotions_df.csv")
df["label"] = df["label"].astype("category")
df["label_id"] = df["label"].cat.codes

unique_labels = df["label"].cat.categories.tolist()

test_df = df.sample(frac=0.2, random_state=42)
test_dataset = Dataset.from_pandas(test_df[["text", "label_id"]].rename(columns={"label_id": "label"}))

# Tokenize with max_length=64
def tokenize_fn(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=64)

tokenized_test = test_dataset.map(tokenize_fn, batched=True, remove_columns=["text"])

# Metrics
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.argmax(preds, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

# TrainingArguments for eval
args = TrainingArguments(
    output_dir="./eval_fp16_shortseq",
    per_device_eval_batch_size=128,
    fp16=True,  # Mixed precision
    do_eval=True,
    report_to="none"
)

# Trainer
trainer = Trainer(
    model=model.to("cuda"),
    tokenizer=tokenizer,
    args=args,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics
)

# Evaluate
results = trainer.evaluate()

# Show
print("\nEvaluation Results:")
for k, v in results.items():
    if isinstance(v, float):
        print(f"{k}: {v:.4f}")


Exception: data did not match any variant of untagged enum ModelWrapper at line 250370 column 3

In [ ]:

# 1. Load both models and tokenizers
goemotions_path = "C:/SonicAid_clean/saved_models/roberta-goemotion-final"
distress_path = "C:/SonicAid_clean/saved_models/distress_classifier"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load models
goemotions_model = AutoModelForSequenceClassification.from_pretrained(goemotions_path).to(device)
goemotions_tokenizer = AutoTokenizer.from_pretrained(goemotions_path)

distress_model = AutoModelForSequenceClassification.from_pretrained(distress_path).to(device)
distress_tokenizer = AutoTokenizer.from_pretrained(distress_path)

# 2. Define emotion label mapping
id2emotion = {
    0: "amusement", 1: "anger", 2: "annoyance", 3: "approval", 4: "caring", 5: "confusion", 6: "curiosity",
    7: "desire", 8: "disappointment", 9: "disapproval", 10: "disgust", 11: "embarrassment", 12: "excitement",
    13: "fear", 14: "gratitude", 15: "grief", 16: "joy", 17: "love", 18: "nervousness", 19: "neutral",
    20: "optimism", 21: "pride", 22: "realization", 23: "relief", 24: "remorse", 25: "sadness", 26: "surprise"
}

# Optional: Categorize emotions
distress_emotions = {"anger", "fear", "grief", "remorse", "sadness", "disappointment", "nervousness"}
non_distress_emotions = {"joy", "love", "gratitude", "optimism", "amusement", "excitement", "surprise"}

# 3. Prediction function
def hybrid_predict(text):
    # Distress model
    dist_input = distress_tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        dist_output = distress_model(**dist_input)
        is_distress = torch.argmax(dist_output.logits, dim=1).item()

    # GoEmotions model
    goemo_input = goemotions_tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        goemo_output = goemotions_model(**goemo_input)
        logits = goemo_output.logits
        probs = torch.softmax(logits, dim=1).cpu().numpy()[0]

    # Top predicted emotions
    top_indices = np.argsort(probs)[::-1][:5]
    top_emotions = [(id2emotion[i], round(probs[i], 4)) for i in top_indices]

    # Filtered emotions
    if is_distress:
        filtered = [(emo, prob) for emo, prob in top_emotions if emo in distress_emotions]
    else:
        filtered = [(emo, prob) for emo, prob in top_emotions if emo in non_distress_emotions]

    return {
        "text": text,
        "distress": bool(is_distress),
        "top_emotions": top_emotions,
        "filtered_emotions": filtered
    }

# 4. Try on sample text
sample = "I feel so hopeless and tired of everything."
result = hybrid_predict(sample)

print("\n Input Text:")
print(result["text"])
print("\n Distress Detected:" if result["distress"] else "\n No Distress Detected:")
print("\nTop 5 Emotions:", result["top_emotions"])
print("Filtered Emotions:", result["filtered_emotions"])


Exception: data did not match any variant of untagged enum ModelWrapper at line 250370 column 3